In [ ]:
!pip install gradio openai

In [ ]:
!pip install gradio openai "deepgram-sdk[async]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os

try:
    drive.mount('/content/drive', force_remount=True) # force_remount untuk memastikan koneksi ulang
    print("✅ Google Drive berhasil di-mount.")
except Exception as e:
    print(f"❌ Gagal me-mount Google Drive. Error: {e}")
    raise

path_model_anda = "/content/drive/MyDrive/NLP_FP/saved_model"


# 3. Verifikasi otomatis untuk memastikan path sudah benar
try:
    print("-" * 50)
    print(f"Mencoba mengakses folder: {path_model_anda}")

    # os.listdir() akan menampilkan semua file di dalam folder 'saved_model'
    isi_folder = os.listdir(path_model_anda)

    print("✅ PATH BENAR! Folder ditemukan.")
    print("Isi folder Anda adalah:")
    for item in isi_folder:
        print(f"- {item}")

except NotADirectoryError:
    print(f"❌ PATH SALAH! Path '{path_model_anda}' menunjuk ke sebuah file, bukan folder. Coba hapus nama file dari akhir path.")
except FileNotFoundError:
    print(f"❌ PATH SALAH! Folder '{path_model_anda}' tidak ditemukan. Periksa kembali nama folder Anda.")
except Exception as e:
    print(f"❌ Terjadi error lain: {e}")

Mounted at /content/drive
✅ Google Drive berhasil di-mount.
--------------------------------------------------
Mencoba mengakses folder: /content/drive/MyDrive/NLP_FP/saved_model
✅ PATH BENAR! Folder ditemukan.
Isi folder Anda adalah:
- config.json
- generation_config.json
- special_tokens_map.json
- model.safetensors
- spiece.model
- tokenizer.json
- tokenizer_config.json


In [ ]:
print("⏳ Membersihkan library lama dan menginstal versi baru...")

# Uninstall library yang mungkin berkonflik terlebih dahulu untuk memastikan instalasi bersih
# Opsi -y adalah untuk menjawab "Yes" secara otomatis
!pip uninstall transformers torch torchvision torchaudio -y

# Install versi yang kita inginkan setelah pembersihan
!pip install gradio "google-generativeai" "transformers[sentencepiece]" "torch" --upgrade --quiet

print("✅ Instalasi bersih selesai. Harap RESTART SESI sekarang.")

⏳ Membersihkan library lama dan menginstal versi baru...
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.5 MB/s eta 0:00:00
   ━━━━

In [ ]:
# Bagian 1: Instalasi & Impor Library
print("⏳ Menginstal library yang diperlukan...")
# DIUBAH: Menambahkan 'deepgram-sdk'
!pip install gradio "google-generativeai" "transformers[sentencepiece]" "torch" "deepgram-sdk" --upgrade --quiet
print("✅ Library selesai diinstal.")

import gradio as gr
import google.generativeai as genai
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from google.colab import drive
from google.colab import userdata
import os
# BARU: Impor library Deepgram
from deepgram import (
    DeepgramClient,
    PrerecordedOptions,
)
import mimetypes # Untuk mendeteksi tipe file audio

# Bagian 2: Setup API Key dan Model Lokal

# --- Setup Google Gemini API ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Kunci API Gemini berhasil dikonfigurasi.")
except Exception as e:
    print("⚠️ PERINGATAN: Kunci API Gemini tidak ditemukan di Colab Secrets. Fungsi Gemini STT tidak akan bekerja.")

# --- BARU: Setup Deepgram API ---
# Kunci API Anda telah ditambahkan di sini.
# Untuk keamanan, disarankan untuk menyimpannya di Colab Secrets dengan nama 'DEEPGRAM_API_KEY'
DEEPGRAM_API_KEY = "DEEPGRAM_API_KEY" # <--- KUNCI ANDA DIMASUKKAN DI SINI
if DEEPGRAM_API_KEY != "DEEPGRAM_API_KEY":
    print("✅ Kunci API Deepgram berhasil dikonfigurasi.")
else:
    print("❌ GAGAL: Harap ganti 'DEEPGRAM_API_KEY' dengan kunci asli Anda.")
    # Coba ambil dari secret jika variabel di atas tidak diubah
    try:
        DEEPGRAM_API_KEY = userdata.get('DEEPGRAM_API_KEY')
        print("✅ Berhasil memuat kunci API Deepgram dari Colab Secrets.")
    except Exception as e:
        print("⚠️ PERINGATAN: Kunci API Deepgram tidak ditemukan. Fungsi Deepgram STT tidak akan bekerja.")

# --- Setup Model Lokal dari Google Drive ---
try:
    print("\n⏳ Menghubungkan ke Google Drive...")
    drive.mount('/content/drive', force_remount=True)

    MODEL_PATH = "/content/drive/MyDrive/NLP_FP/saved_model"

    print(f"⏳ Memuat model koreksi grammar dari: {MODEL_PATH}")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Menggunakan device: {device}")

    tokenizer_local = AutoTokenizer.from_pretrained(MODEL_PATH)
    model_local = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(device)
    print("✅ Model koreksi grammar lokal berhasil dimuat.")
except Exception as e:
    print(f"❌ GAGAL memuat model dari Drive. Pastikan path '{MODEL_PATH}' benar. Error: {e}")
    raise

# Bagian 3: Definisi Fungsi-Fungsi Inti

# --- Fungsi untuk Speech-to-Text (STT) ---

def speech_to_text_gemini(audio_path):
    """Mengirim file audio ke Gemini API dan mengembalikan hasil transkripsi."""
    print(f"\n Mentraskripsikan audio '{os.path.basename(audio_path)}' menggunakan Gemini...")
    if not GEMINI_API_KEY:
      return "Error: Kunci API Gemini tidak dikonfigurasi."
    try:
        gemini_model_stt = genai.GenerativeModel('models/gemini-1.5-flash-latest')
        audio_file = genai.upload_file(path=audio_path)
        response = gemini_model_stt.generate_content(
            ["Transcribe this audio accurately.", audio_file],
            request_options={"timeout": 600}
        )
        print("✅ Transkripsi Gemini berhasil.")
        return response.text.strip()
    except Exception as e:
        print(f"❌ Error saat transkripsi dengan Gemini: {e}")
        return f"Error Gemini: {e}"

# BARU: Fungsi untuk STT menggunakan Deepgram
def speech_to_text_deepgram(audio_path):
    """Mengirim file audio ke Deepgram API dan mengembalikan hasil transkripsi."""
    print(f"\n Mentraskripsikan audio '{os.path.basename(audio_path)}' menggunakan Deepgram...")
    if not DEEPGRAM_API_KEY or DEEPGRAM_API_KEY == "DEEPGRAM_API_KEY":
      return "Error: Kunci API Deepgram tidak dikonfigurasi."
    try:
        # Inisialisasi Deepgram Client
        deepgram = DeepgramClient(DEEPGRAM_API_KEY)

        with open(audio_path, "rb") as file:
            buffer_data = file.read()

        payload = {"buffer": buffer_data}

        # Konfigurasi opsi transkripsi
        options = PrerecordedOptions(
            model="nova-2",      # Model yang canggih
            smart_format=True,   # Format cerdas (kapitalisasi, pungtuasi)
            language="en-US",    # Bahasa Inggris
        )

        # Kirim permintaan transkripsi
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
        transcript = response["results"]["channels"][0]["alternatives"][0]["transcript"]

        print("✅ Transkripsi Deepgram berhasil.")
        return transcript

    except Exception as e:
        print(f"❌ Error saat transkripsi dengan Deepgram: {e}")
        return f"Error Deepgram: {e}"

# --- Fungsi untuk Koreksi Grammar menggunakan Model Lokal ---
def koreksi_grammar_lokal(teks_asli):
    """Melakukan koreksi grammar menggunakan model dari Drive."""
    print("✍️ Melakukan koreksi grammar dengan model lokal...")
    try:
        input_text = f"grammar: {teks_asli}"
        inputs = tokenizer_local(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model_local.generate(inputs.input_ids, max_length=512, num_beams=5, early_stopping=True)
        corrected_text = tokenizer_local.decode(outputs[0], skip_special_tokens=True)
        print("✅ Koreksi grammar lokal berhasil.")
        return corrected_text
    except Exception as e:
        print(f"❌ Error saat koreksi dengan model lokal: {e}")
        return f"Error Model Lokal: {e}"

# --- DIUBAH: Fungsi Utama yang dipanggil oleh Gradio ---
def process_audio_and_correct(stt_provider, audio_path):
    """
    Fungsi utama yang mengorkestrasi alur kerja:
    1. Memilih mesin STT (Gemini atau Deepgram).
    2. Melakukan transkripsi.
    3. Melakukan koreksi grammar.
    4. Mengembalikan output untuk ditampilkan di Gradio dengan label dinamis.
    """
    if audio_path is None:
        # Jika tidak ada audio, kembalikan komponen output yang kosong dan tidak terlihat
        return gr.Textbox(value="", visible=False), gr.Textbox(value="", visible=False)

    # Langkah 1: Transkripsi audio berdasarkan pilihan provider
    if stt_provider == "Deepgram":
        transcribed_text = speech_to_text_deepgram(audio_path)
    else: # Default ke Gemini
        transcribed_text = speech_to_text_gemini(audio_path)

    if not transcribed_text or "Error" in transcribed_text:
        # Jika ada error, tampilkan hanya di box pertama
        return gr.Textbox(value=transcribed_text, label=f"Error dari {stt_provider}", visible=True), gr.Textbox(value="", visible=False)

    # Langkah 2: Koreksi grammar dari hasil transkripsi
    corrected_text = koreksi_grammar_lokal(transcribed_text)

    # Langkah 3: Kembalikan komponen Gradio untuk memperbarui UI
    # Ini memungkinkan kita mengubah 'label' dan 'value' secara dinamis
    return (
        gr.Textbox(value=transcribed_text, label=f"Teks Asli (Dengan {stt_provider})", visible=True),
        gr.Textbox(value=corrected_text, label="Teks yang Diperbaiki (Dengan Model Kami)", visible=True)
    )

# Bagian 4: Luncurkan Antarmuka Gradio

print("\n🚀 Meluncurkan antarmuka Gradio...")

# DIUBAH: Antarmuka Gradio dengan pilihan STT
iface = gr.Interface(
    fn=process_audio_and_correct,
    inputs=[
        gr.Radio(
            ["Deepgram", "Gemini"],
            label="Pilih Mesin Speech-to-Text",
            value="Deepgram" # Nilai default
        ),
        gr.Audio(
            sources=["microphone"],
            type="filepath",
            label="Rekam Suara Anda (Bahasa Inggris)"
        )
    ],
    outputs=[
        # Output akan dibuat tidak terlihat pada awalnya, dan akan diisi oleh fungsi `process_audio_and_correct`
        gr.Textbox(label="Teks Asli (Hasil Transkripsi)", visible=False),
        gr.Textbox(label="Teks yang Diperbaiki (Dengan Model Kami)", visible=False)
    ],
    title="Speech-to-Text & English Grammar Correction",
    description="Rekam suara Anda dalam Bahasa Inggris. Pilih mesin (Deepgram atau Gemini) untuk transkripsi, lalu hasilnya akan diperbaiki oleh model grammar.",
    allow_flagging="never"
)

iface.launch(debug=True)

⏳ Menginstal library yang diperlukan...
✅ Library selesai diinstal.
✅ Kunci API Gemini berhasil dikonfigurasi.
❌ GAGAL: Harap ganti 'DEEPGRAM_API_KEY' dengan kunci asli Anda.
✅ Berhasil memuat kunci API Deepgram dari Colab Secrets.

⏳ Menghubungkan ke Google Drive...
Mounted at /content/drive
⏳ Memuat model koreksi grammar dari: /content/drive/MyDrive/NLP_FP/saved_model
Menggunakan device: cuda
✅ Model koreksi grammar lokal berhasil dimuat.

🚀 Meluncurkan antarmuka Gradio...


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:419: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://93b860fcde268a5b8d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Gemini...
✅ Transkripsi Gemini berhasil.
✍️ Melakukan koreksi grammar dengan

/tmp/ipython-input-5-1804487448.py:109: UnsupportedWarning: prerecorded is unsupported as of 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Deepgram...
✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Deepgram...
✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Deepgram...
✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Deepgram...
✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lokal berhasil.

 Mentraskripsikan audio 'audio.wav' menggunakan Deepgram...
✅ Transkripsi Deepgram berhasil.
✍️ Melakukan koreksi grammar dengan model lokal...
✅ Koreksi grammar lok

In [ ]:
!ls -l /content/drive/MyDrive/NLP_FP/saved_model

total 873829
-rw------- 1 root root      1565 Jun  1 14:15 config.json
-rw------- 1 root root       149 Jun  1 14:15 generation_config.json
-rw------- 1 root root 891558696 Jun  1 14:16 model.safetensors
-rw------- 1 root root      2308 Jun  1 14:16 special_tokens_map.json
-rw------- 1 root root    791656 Jun  1 14:16 spiece.model
-rw------- 1 root root     21683 Jun  1 14:16 tokenizer_config.json
-rw------- 1 root root   2422191 Jun  1 14:16 tokenizer.json
